In [ ]:
import cleverhans
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# new Data preparation

# Training Dataset
data = pd.read_csv("./dataset/UNSW_2018_IoT_Botnet_Final_10_best_Training.csv")

x_features = data[['seq','stddev','N_IN_Conn_P_SrcIP', 'min', 'state_number', 'mean', 'N_IN_Conn_P_DstIP',
       'drate', 'srate', 'max', 'category', 'subcategory']]

# Label Encoding the target columns
le = LabelEncoder()
x_features['category'] = le.fit_transform(x_features['category'])
x_features['subcategory'] = le.fit_transform(x_features['subcategory'])

x = x_features.values
x = preprocessing.scale(x)
y = data['attack'].values

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

y_train_cat = np_utils.to_categorical(y_train)
y_test_cat = np_utils.to_categorical(y_test)

In [ ]:
# Define model

model = tf.keras.Sequential([
        tf.keras.layers.Dense(200, input_dim=x.shape[1], activation=tf.nn.relu),
        tf.keras.layers.Dense(500, activation=tf.nn.relu),
        tf.keras.layers.Dense(200, activation=tf.nn.relu),
        tf.keras.layers.Dense(len(np.unique(y))),
        # We seperate the activation layer to be able to access
        # the logits of the previous layer later
        tf.keras.layers.Activation(tf.nn.softmax)
        ])

model.compile(optimizer='adam',
              loss= 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train Classification model

from keras.utils import np_utils
y_train_cat = np_utils.to_categorical(y_train)
y_test_cat = np_utils.to_categorical(y_test)

model.fit(X_train, y_train_cat, epochs=10, validation_split=0.2)

test_loss, test_acc = model.evaluate(X_test, y_test_cat)
print('Test accuracy:', test_acc)

In [ ]:
# Original Confusion Matrix of Model

from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
y_pred=np.argmax(y_pred, axis=1)
cm_org = confusion_matrix(y_test, y_pred)
print("*"*50)
print("Original confusion matrix")
print(cm_org)

In [ ]:
# Import the attack
from cleverhans.future.tf2.attacks import fast_gradient_method

#The attack requires the model to ouput the logits
logits_model = tf.keras.Model(model.input, model.layers[-1].output)

epsilon = 0.1
adv_fgsm_x = fast_gradient_method(logits_model, X_test, epsilon, np.inf, targeted=False)

# Attack sample confusion matrix
y_pred = model.predict(adv_fgsm_x)
y_pred=np.argmax(y_pred, axis=1)
cm_adv = confusion_matrix(y_test, y_pred)
print("*"*20)
print("Attacked confusion matrix")
print(cm_adv)

In [ ]:
# Adversarial sample generated
print(adv_fgsm_x)

In [ ]:
# Adversarial training confusion matrix
adv_fgsm_x_train = fast_gradient_method(logits_model, X_train, epsilon, np.inf, targeted=False)
adv_fgsm_x_test = fast_gradient_method(logits_model, X_test, epsilon, np.inf, targeted=False)

concat_adv_x = np.concatenate([X_train, adv_fgsm_x_train])
concat_y_train = np.concatenate([y_train_cat, y_train_cat])

# Adversarial training
model.fit(concat_adv_x, concat_y_train, epochs=10, validation_data=(adv_fgsm_x_test, y_test_cat))
y_pred = model.predict(adv_fgsm_x_test)

# Print out Confusion Matrix
cm_adv = confusion_matrix(y_test, y_pred)
print("*"*20)
print("Adversarial training confusion matrix")
print(cm_adv)